<img src="images/borehole_graph.png" ></img>

<br>

# Borehole Graph Data Model Example

In this notebook, we will use <a href="https://steno3d.com">Steno3D</a> to view and share data throughout the steps of a lithological borehole graph from neo4j. 

## Query the data from Neo4j using Neo4j Driver

First step is to use a cypher query to extract the data from Neo4j

In [12]:
from neo4j import GraphDatabase

uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

def borehole_data(tx):
    for record in tx.run("match (c:Collar)-[:NEXT_INTERVAL*]->(i:Interval) "
                         "return c.Name,i.Pri_Material,i.TopDepth,i.location.x,i.location.y,i.location.z order by c.Name,i.ToDepth"):
        print(record["c.Name"])
        
with driver.session() as session:
    session.read_transaction(borehole_data)

1
1
1
1
1
100
100
100
100
101
101
101
101
102
102
102
102
103
103
103
103
104
104
104
104
104
105
105
106
106
106
106
106
108
108
108
108
109
109
109
11
11
11
11
11
11
110
110
111
111
111
111
111
112
112
113
113
113
114
114
114
114
115
115
116
116
116
116
117
117
118
118
119
119
119
119
12
12
12
12
120
120
121
121
122
122
122
123
123
124
124
124
124
125
125
125
125
126
126
126
127
127
127
127
128
128
128
128
132
132
132
132
132
132
132
132
132
133
133
133
133x
133x
133x
133x
133x
134
134
134
135
135
136
136
137
137
137
137
138
138
139
139
139
140
140
140
140
141
141
141
141
141
141
141
141
142
142
142
142
143
143
143
143
144
144
144
144
144
145
145
145
145
145
145
145
145
146
146
147
147
147
148
148
149
149
149
149
15
15
15
15
15
15
15
150
150
150
150
151
151
151
152_redrill
152_redrill
152_redrill
152_redrill
153
153
153
153
153
155
155
155
155
156
156
157
157
157
158
158
159
159
159
160
160
160
161
161
162
162
163
163
163
163
164
164
164
164
164
165
165
165
165
165
165
166
166
166
16

## Login to Steno3D

The first thing you need to do is login to Steno3D. You can <a href="https://steno3d.com/signup">sign up for an account</a> to get your own developer API key if you do not have one already. Running the cell below will provide you with instructions for how to obtain and enter your key.

In [ ]:
steno3d.login()

I think this is markdown but who knoes

# Hello

*I and italic*

For this notebook, we will be using example data from the `steno3d.examples.Wolfpass` module. This module downloads raw data, saves it to a folder in the home directory (or another location you may specify), and allows easy access to all the different pieces of data.

In [ ]:
from steno3d.examples import Wolfpass
Wolfpass.fetch_data(verbose=False)  # To specify a different data directory than the
                                    # default '$HOME/.steno3d_python_assets/' use
                                    # keyword argument `directory='/path/to/your/directory/'`

## Inspect the Surface Topography

The first step of using Steno3D is to create a <a href="https://python.steno3d.com/en/latest/content/api/projects.html">project</a>. Let's make one that will include all our preliminary data used to plan the mineral resource exploration. We will create a different project later when we have some results to display.

This project is public, so anyone can view it once it is uploaded. When working with your own data, you may want to create private projects to share only with your collaborators. Different Steno3D accounts allow different numbers of private projects; <a href="https://steno3d.com/pricing">learn about these accounts and how to upgrade on the Steno3D website</a>.

In [ ]:
prelim_data = steno3d.Project(
    title='Wolf Pass Exploration: Preliminary Data',
    description='This project will contain surface topography and planned borehole drill locations.',
    public=True
)

Now we can start to construct our ground surface. To build the <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#meshes">mesh</a>, we use vertices and triangles imported from our `Wolfpass` example module.

In [ ]:
ground_mesh = steno3d.Mesh2D(
    vertices=Wolfpass.topo_vertices,
    triangles=Wolfpass.topo_triangles
)

Elevation data will be useful; this can come directly from the `vertices`.

In [ ]:
ground_elev = steno3d.DataArray(
    title='Ground Surface Elevation',
    description='Elevation values correspond to mesh vertices',
    array=Wolfpass.topo_vertices[:,2]  # Elevation corresponds to Z-component of vertices
)

We also have some surface imagery of the area. The image and image orientation are avilable in the `Wolfpass` example module. They can be applied as a <a href="https://python.steno3d.com/en/latest/content/api/resources/texture.html#resources-texture">texture</a>.

In [ ]:
ground_image = steno3d.Texture2DImage(
    title='Surface Imagery',
    image=Wolfpass.topo_image,
    O=Wolfpass.topo_image_orientation['O'],
    U=Wolfpass.topo_image_orientation['U'],
    V=Wolfpass.topo_image_orientation['V']
)

We can now add these components to our ground surface. (For tips on the data binding step, refer to <a href="https://python.steno3d.com/en/latest/content/api/resources/surface.html#data">binding data to surface</a>.

In [ ]:
ground_surf = steno3d.Surface(
    project=prelim_data,
    title='Ground Surface',
    description='Surface with elevation data and surface imagery.',
    mesh=ground_mesh,
    textures=ground_image,
    data=[dict(location='N',
              data=ground_elev)]
)

We can now upload this surface to <a href="https://steno3d.com">steno3d.com</a> and view the surface here in the notebook to ensure it looks okay.

In [ ]:
ground_surf.upload()
ground_surf.plot()

## Decide on Drill Locations

Now that we have the ground surface, we need to add some proposed drill location points. First, we need a <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html#meshes">mesh 0D</a>. The vertex data is imported from the `Wolfpass` module.

In [ ]:
drill_locations = steno3d.Mesh0D(
    vertices=Wolfpass.drill_vertices
)

We only need the locations of these points, no other data, so let's construct our <a href="https://python.steno3d.com/en/latest/content/api/resources/point.html">point</a> resource now. We will make the points orange so they stand out when plotted with the surface imagery.

In [ ]:
drill_points = steno3d.Point(
    project=prelim_data,
    title='Proposed Drill Locations',
    mesh=drill_locations,
    opts=dict(color='orange')
)

After uploading these points to our project on <a href="https://steno3d.com">steno3d.com</a>, let's print the url of our project to share.

In [ ]:
drill_points.upload()

In [ ]:
print(prelim_data.url)

## Plot the Borehole Data

After discussing the preliminary data project online with collaborators, it was decided to go ahead with borehole drilling. Now the data is back and available to be explored with. Let's make a new project to contain the results of the drilling.

In [ ]:
result_data = steno3d.Project(
    title='Wolf Pass Exploration: Results',
    description='This project will contain borehole data and interpreted formation surface.',
    public=True  # Public for our example, since private quota is limited.
)

To display our borehole data, we first need a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html#meshes">mesh 1D</a> based on their spatial geometry. This is imported from the `Wolfpass` module. The `view_type` option sets the lines mesh to initially display as fixed-width, extruded "tubes" rather than true 1D lines.

In [ ]:
borehole_mesh = steno3d.Mesh1D(
    vertices = Wolfpass.borehole_vertices,
    segments = Wolfpass.borehole_segments,
    opts = dict(
        view_type='boreholes'
    )
)

Let's create a <a href="https://python.steno3d.com/en/latest/content/api/resources/line.html">line</a> resource now using this mesh.

In [ ]:
boreholes = steno3d.Line(
    project=result_data,
    title='Borehole Results',
    description='Data based on results of borehole drilling',
    mesh=borehole_mesh
)

We need to add all the data to the boreholes now.

In [ ]:
borehole_raw_data = Wolfpass.borehole_data
borehole_data = [
    steno3d.DataArray(title=rd, array=borehole_raw_data[rd]) for rd in borehole_raw_data
]
borehole_bound_data = [
    dict(location='CC', data=d) for d in borehole_data
]

In [ ]:
boreholes.data = borehole_bound_data

These boreholes will be easier to analyze if the ground surface is also present. We can simply add the ground surface resource created for the <a href="#Inspect-the-Surface-Topography">preliminary data project</a> to our results data project.

In [ ]:
result_data.resources += [ground_surf]

We can adjust the opacity of this surface so the boreholes are not obscured.

In [ ]:
ground_surf.opts.opacity = 0.25

Now our borehole results can be uploaded and shared.

In [ ]:
result_data.upload()

## Explore the Modeled Formation

Based on the borehole results, modelling of the dacite formation has been carried out. Let's view this modeled surface with our other results using <a href="https://steno3d.com">Steno3D</a>. We will create, upload, and print the url for sharing in one step.

In [ ]:
dacite_ind = Wolfpass.lith_names.index('dacite')
dacite_vertices = Wolfpass.lith_vertices[dacite_ind]
dacite_triangles = Wolfpass.lith_triangles[dacite_ind]

dacite_surface = steno3d.Surface(
    project=result_data,
    title='Dacite Formation',
    description='Bounding surface of dacite based on borehole data and modelling results',
    mesh=steno3d.Mesh2D(
        vertices=dacite_vertices,
        triangles=dacite_triangles
    ),
    opts=dict(
        color='red'
    )
)
dacite_surface.upload()
print(result_data.url)

___
**Navigation**
- <a href="#Comprehensive-Example:-Wolf-Pass-Exploration-Project">Top of page</a>
- <a href="index.ipynb">Notebook home</a>
- <a href="https://steno3d.com">Steno3D website</a>
- <a href="https://steno3d.com/docs">Steno3D documentation</a>
- <a href="https://github.com/seequent/steno3d-notebooks/issues/new">Report an issue</a>
___